In [154]:
from core.goodreads import GoodReadsData
import gc

goodreads = GoodReadsData()
filename = goodreads.file_names[0]
filename

'goodreads_book_works'

In [155]:
gc.collect()

1310

# Análisis de `goodreads_book_works`

In [156]:
import os
import pandas as pd
import numpy as np

In [157]:
# Download
if not os.path.exists(goodreads.get_file_path(filename)):
    goodreads.download_file(filename)
    
# Load
df = goodreads.load_file(filename)

In [158]:
def get_limits(c: str) -> tuple[int,int]:
    return (df.loc[df[c]!="", c].astype(np.int64).min(),df.loc[df[c]!="", c].astype(np.int64).max())

### Análisis
1. Comprender datos.
2. Detectar valores ausentes.
3. Detectar duplicados
4. DEtectar valores erróneos.

In [159]:
df.sample(5)

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
3535,15,15565,10,,404,300388,2007,Candy Cane Murder,5:2928|4:2881|3:2513|2:537|1:133|total:8992,,1,,8992,book,34910,1582879
1006242,2,49,,,3,2070645,1961,"England, Half English: A Polyphoto of the Fifties",5:3|4:9|3:7|2:1|1:0|total:20,,,,20,book,74,2075888
1258271,2,11,12,,1,13283281,2011,,5:3|4:2|3:2|2:0|1:0|total:7,,14,,7,book,29,18487738
1353325,3,99,6,,3,8827248,2010,,5:4|4:4|3:0|2:0|1:0|total:8,,21,,8,book,36,13701613
1057493,1,17,9,,3,16061701,2012,,5:9|4:2|3:1|2:0|1:1|total:13,,30,,13,book,57,21849745


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521962 entries, 0 to 1521961
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype 
---  ------                             --------------    ----- 
 0   books_count                        1521962 non-null  int64 
 1   reviews_count                      1521962 non-null  int64 
 2   original_publication_month         1521962 non-null  object
 3   default_description_language_code  1521962 non-null  object
 4   text_reviews_count                 1521962 non-null  int64 
 5   best_book_id                       1521962 non-null  int64 
 6   original_publication_year          1521962 non-null  object
 7   original_title                     1521962 non-null  object
 8   rating_dist                        1521962 non-null  object
 9   default_chaptering_book_id         1521962 non-null  object
 10  original_publication_day           1521962 non-null  object
 11  original_language_id               15

A priori no existen valores `nan`, sin embargo es posible que estos estén ocultos.

In [161]:
(df == "").sum()

books_count                                0
reviews_count                              0
original_publication_month            486516
default_description_language_code    1521962
text_reviews_count                         0
best_book_id                               0
original_publication_year             252575
original_title                        875056
rating_dist                                0
default_chaptering_book_id           1521302
original_publication_day              562502
original_language_id                 1521962
ratings_count                              0
media_type                            414096
ratings_sum                                0
work_id                                    0
dtype: int64

Aquí si vemos como algunas columnas sí tienen valores `nan` ocultos para `original_publication_month`, `default_description_language_code`, `original_publication_year`, `original_title`, `default_chaptering_book_id`, `original_publication_day`, `original_language_id` y `media_type`.

In [162]:
df.nunique()

books_count                              628
reviews_count                          27553
original_publication_month                13
default_description_language_code          1
text_reviews_count                      4794
best_book_id                         1521962
original_publication_year               1264
original_title                        611914
rating_dist                           691714
default_chaptering_book_id               661
original_publication_day                  32
original_language_id                       1
ratings_count                          19463
media_type                                 5
ratings_sum                            37952
work_id                              1521962
dtype: int64

In [163]:
df["original_publication_month"].unique()

array(['8', '', '7', '11', '4', '1', '6', '3', '9', '5', '2', '12', '10'],
      dtype=object)

In [164]:
df["default_description_language_code"].unique()

array([''], dtype=object)

In [165]:
df["original_publication_day"].unique()

array(['', '14', '1', '10', '25', '22', '20', '23', '4', '3', '16', '7',
       '17', '27', '9', '24', '31', '2', '30', '5', '21', '6', '29', '12',
       '13', '18', '11', '26', '28', '15', '19', '8'], dtype=object)

In [166]:
df["original_language_id"].unique()

array([''], dtype=object)

In [167]:
df["media_type"].unique()

array(['book', '', 'not a book', 'periodical', 'article'], dtype=object)

De las anteriores, `original_description_language_code` y `original_language_id` contienen sólo valores nulos, por lo que se pueden eliminar las columnas enteras. Por otro lado, `original_publication_month` y `original_publication_day` contienen los valores correspondientes a los meses del año (del 1 al 12) y a los días del mes (del 1 al 31) respectivamente, añadiendo los valores ausentes `""`. Por último, `media_type` contiene 4 tipos de categorías + los valores ausentes.

In [168]:
df.duplicated(["best_book_id"]).sum()

np.int64(0)

In [169]:
df.duplicated(["work_id"]).sum()

np.int64(0)

No existen identificadores duplicados y, por tanto, registros duplicados.

In [170]:
df.describe()

,books_count,reviews_count,text_reviews_count,best_book_id,ratings_count,ratings_sum,work_id
count,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06
mean,4.554304e+00,1.228961e+03,4.451360e+01,1.498071e+07,6.440522e+02,2.574487e+03,2.383090e+07
std,1.719140e+01,2.073292e+04,5.673989e+02,1.120875e+07,1.479944e+04,6.095401e+04,1.946726e+07
min,1.000000e+00,-6.069000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,4.000000e+01
25%,1.000000e+00,1.800000e+01,2.000000e+00,3.185279e+06,7.000000e+00,2.600000e+01,3.247694e+06
50%,2.000000e+00,7.600000e+01,5.000000e+00,1.572043e+07,2.600000e+01,1.010000e+02,2.137204e+07
75%,4.000000e+00,3.270000e+02,1.500000e+01,2.456787e+07,1.100000e+02,4.230000e+02,4.386945e+07
max,3.676000e+03,6.057595e+06,1.565750e+05,3.661954e+07,5.066596e+06,2.211059e+07,5.837948e+07


Observamos como `reviews_count` contiene valores negativos, lo cual se considera un dato erróneo.

In [171]:
(df["reviews_count"] < 0).sum()

np.int64(46)

In [172]:
df[df["reviews_count"] < 0].sort_values("reviews_count").head(5)

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
224794,4,-6069,10,,41,27157487,2012,,5:46|4:24|3:13|2:1|1:0|total:84,,28,,84,,367,56535531
732048,3,-3494,5,,37,19227803,2012,,5:91|4:89|3:104|2:27|1:25|total:336,,11,,336,,1202,57148033
233901,3,-3479,,,61,25635519,,,5:320|4:155|3:49|2:6|1:1|total:531,,,,531,book,2380,56535888
8448,4,-1306,10,,113,9456835,2008,,5:117|4:228|3:161|2:50|1:24|total:580,,14,,580,book,2104,54854952
284931,5,-718,10,,74,34811889,2014,,5:162|4:149|3:71|2:13|1:4|total:399,,27,,399,book,1649,56577769


Suponiendo que este valor no puede ser negativo, trasnformaremos estos valores a su valor absoluto. Es posible que ocurra algo similar en otras columnas que parecen numéricas.

In [173]:
df[df["default_chaptering_book_id"]!=""].sample(5)

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
144241,24,2787,1,,205,22730614,2015,,5:350|4:381|3:323|2:73|1:22|total:1149,22608859,27,,1149,book,4411,42266464
1937,991,255567,,,4066,16343,1922,,5:54097|4:58743|3:34917|2:5871|1:3469|total:15...,16343,,,157097,book,625419,3366260
211122,6,6472,,,297,17427430,2013,,5:362|4:616|3:542|2:212|1:53|total:1785,22845365,,,1785,book,6377,44846065
861922,4,42,10,,3,16088742,2012,Hexual Healing,5:5|4:4|3:5|2:1|1:1|total:16,24972717,12,,16,book,59,21916146
1481766,2,12,4,,1,17793014,2013,,5:2|4:3|3:0|2:0|1:0|total:5,17793014,3,,5,book,22,24887718


In [174]:
for column in ["original_publication_year", "default_chaptering_book_id"]:
    
    min, max = get_limits(column)
    print(f"{column}: min={min}, max={max}")

original_publication_year: min=-2600, max=32767
default_chaptering_book_id: min=1, max=28572442


In [175]:
(df.loc[df["original_publication_year"]!="", "original_publication_year"].astype(np.int64)<0).sum()

np.int64(354)

In [176]:
df.loc[
    df[df["original_publication_year"] != ""]
    .assign(original_publication_year=lambda d: d["original_publication_year"].astype(np.int64))
    .sort_values("original_publication_year")
    .iloc[300:310].index
]

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
1493797,15,224,,,10,551650,-54,,5:48|4:30|3:16|2:3|1:1|total:98,,,,98,book,415,18837432
1505565,21,183,,,6,400299,-51,Somnium Scipionis,5:25|4:33|3:22|2:3|1:0|total:83,,,,83,book,329,389724
127879,27,4577,,,57,414078,-51,De Re Publica et De Legibus,5:531|4:534|3:458|2:110|1:18|total:1651,,,,1651,book,6403,3210395
939407,41,650,,,18,167907,-50,Commentarii de Bello Gallico / Commentarii de ...,5:63|4:100|3:55|2:11|1:3|total:232,,,,232,book,905,162168
310678,403,14236,,,304,592167,-50,Commentarii de Bello Gallico,5:2239|4:2456|3:1552|2:342|1:73|total:6662,,,,6662,book,26432,1978328
1427658,15,1397,,,26,615103,-50,,5:135|4:118|3:84|2:25|1:6|total:368,,,,368,book,1455,601543
318250,55,1248,,,31,37807,-50,,5:199|4:209|3:96|2:18|1:2|total:524,,,,524,book,2157,3396563
751351,4,6,,,1,1847706,-50,Pro Rabirio Postumo,5:1|4:0|3:2|2:0|1:0|total:3,,,,3,book,11,1848134
556960,1,48,,,3,23505892,-50,,5:0|4:5|3:13|2:5|1:0|total:23,,,,23,book,69,50286499
100715,101,8783,,,74,56781,-47,De bello civili,5:1885|4:1664|3:1122|2:227|1:48|total:4946,,,,4946,book,19949,162131


Observando que existían valores negativos en `original_publication_year`, se intuía que dichos valores eran erróneos, por lo que tras checkear en la página web algunos ejemplos, hemos deducido que el valor correcto se corresponde con el valor absoluto de dicho valor. Lo corregiremos más adelante, aún así, es posible que realicemos una comprobación mediante la API.

### Transformación
1. Eliminamos columnas inútiles.
2. Transformamos valores erróneos.
3. Parsear `rating_dist`.
4. Transformar columnas.
5. Añadir valores `nan`.

In [177]:
df.drop(columns=["default_description_language_code", "original_language_id", "ratings_sum"], inplace=True)

In [179]:
df["reviews_count"] = df["reviews_count"].abs()

In [180]:
def parse_ratings_dist(s: str):
    one_hot = {}
    for r in s.split("|"):
        k, v = r.split(":")
        one_hot[f"n_ratings_{k}"] = int(v)
    return pd.Series(one_hot)

df = pd.concat([df, df["rating_dist"].apply(parse_ratings_dist)], axis=1)

Comprobamos que la nueva columna `n_ratings_total` coincide con la anterior `ratings_count`.

In [181]:
(df["n_ratings_total"] != df["ratings_count"]).sum()

np.int64(0)

Eliminamos la columna original (deprecada) y visualizamos estadísticas de las nuevas.

In [182]:
df.drop(columns=["ratings_count", "rating_dist"], inplace=True)
df[["n_ratings_1", "n_ratings_2", "n_ratings_3", "n_ratings_4", "n_ratings_5", "n_ratings_total"]].describe()

,n_ratings_1,n_ratings_2,n_ratings_3,n_ratings_4,n_ratings_5,n_ratings_total
count,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06,1.521962e+06
mean,1.423870e+01,3.562033e+01,1.320622e+02,2.178336e+02,2.442973e+02,6.440522e+02
std,5.590809e+02,8.437659e+02,2.549407e+03,4.581209e+03,6.936518e+03,1.479944e+04
min,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00
50%,0.000000e+00,2.000000e+00,6.000000e+00,8.000000e+00,8.000000e+00,2.600000e+01
75%,3.000000e+00,7.000000e+00,2.600000e+01,3.600000e+01,3.400000e+01,1.100000e+02
max,4.638080e+05,4.448880e+05,8.087530e+05,1.519190e+06,3.131920e+06,5.066596e+06


Vemos cómo existen valores negativos tanto en `n_ratings_1`, como en `n_ratings_2`. Esto es un dato erróneo que corregiremos.

In [183]:
((df["n_ratings_1"]<0) | (df["n_ratings_5"]<0)).sum()

np.int64(2)

In [184]:
df[(df["n_ratings_1"]<0) | (df["n_ratings_5"]<0)].sample(2)

,books_count,reviews_count,original_publication_month,text_reviews_count,best_book_id,original_publication_year,original_title,default_chaptering_book_id,original_publication_day,media_type,work_id,n_ratings_5,n_ratings_4,n_ratings_3,n_ratings_2,n_ratings_1,n_ratings_total
285474,4,1582,,20,479188,2001,The Grail Quest: The Archers Tale - Vagabond -...,,,book,467605,610,400,104,13,-1,1126
748273,2,20,,2,17411236,2012,The New Way to Make Money in Property,,,book,24255210,-1,0,7,0,0,6


Al igual que ocurría anteriormente con `original_publication_year`, intuímos que el valor correcto será el valor absoluto del mismo. Observamos como este valor erróneo provoca error en `n_ratings_total` por lo que debemos corregirlo también. 

In [185]:
for index in {748273, 285474}:
    df.loc[index, "n_ratings_1"] = abs(df.loc[index, "n_ratings_1"])
    df.loc[index, "n_ratings_5"] = abs(df.loc[index, "n_ratings_5"])
    
    df.loc[index, "n_ratings_total"] = \
        df.loc[index, "n_ratings_1"] + df.loc[index, "n_ratings_2"] + df.loc[index, "n_ratings_3"] \
        + df.loc[index, "n_ratings_4"] + df.loc[index, "n_ratings_5"]
        
df.loc[[748273, 285474]].head()

,books_count,reviews_count,original_publication_month,text_reviews_count,best_book_id,original_publication_year,original_title,default_chaptering_book_id,original_publication_day,media_type,work_id,n_ratings_5,n_ratings_4,n_ratings_3,n_ratings_2,n_ratings_1,n_ratings_total
748273,2,20,,2,17411236,2012,The New Way to Make Money in Property,,,book,24255210,1,0,7,0,0,8
285474,4,1582,,20,479188,2001,The Grail Quest: The Archers Tale - Vagabond -...,,,book,467605,610,400,104,13,1,1128


Corregimos posibles errores en los valores y guardamos los índices de los valores `nan` ocultos para asignar este valor más tarde. 

In [186]:
df["original_publication_month"] = df["original_publication_month"].str.replace(r"\s+", "", regex=True)
df["original_publication_day"] = df["original_publication_day"].str.replace(r"\s+", "", regex=True)
df["original_publication_year"] = df["original_publication_year"].str.replace(r"\s+", "", regex=True)

df["original_title"] = df["original_title"].str.replace(r"\s+", "", regex=True)
df["default_chaptering_book_id"] = df["default_chaptering_book_id"].str.replace(r"\s+", "", regex=True)

df["media_type"] = df["media_type"].str.strip()

In [187]:
opm_nan = df["original_publication_month"] == ""
opd_nan = df["original_publication_day"] == ""
opy_nan = df["original_publication_year"] == ""

ot_nan = df["original_title"] == ""
dcbi_nan = df["default_chaptering_book_id"] == ""

mt_nan = df["media_type"] == ""

Adaptamos valores de columnas para corregir posibles errores y poder hacer la transformación.

In [188]:
df.loc[opm_nan, "original_publication_month"] = 0
df.loc[opd_nan, "original_publication_day"] = 0
df.loc[opy_nan, "original_publication_year"] = 0

df.loc[dcbi_nan, "default_chaptering_book_id"] = 0

In [189]:
df["original_publication_year"] = df["original_publication_year"].astype(np.int64).abs()

Visualizamos valores mínimos y máximos para optimizar la transformación.

In [190]:
for column in [
    "books_count", "reviews_count", "text_reviews_count", "best_book_id", "work_id",
    "n_ratings_1", "n_ratings_2", "n_ratings_3", "n_ratings_4", "n_ratings_5", "n_ratings_total",
    "original_publication_year", "original_publication_month", "original_publication_day",
    "default_chaptering_book_id"]:
    
    min, max = get_limits(column)
    print(f"{column}: min={min}, max={max}")

books_count: min=1, max=3676
reviews_count: min=0, max=6057595
text_reviews_count: min=0, max=156575
best_book_id: min=1, max=36619543
work_id: min=40, max=58379481
n_ratings_1: min=0, max=463808
n_ratings_2: min=0, max=444888
n_ratings_3: min=0, max=808753
n_ratings_4: min=0, max=1519190
n_ratings_5: min=0, max=3131920
n_ratings_total: min=0, max=5066596
original_publication_year: min=0, max=32767
original_publication_month: min=0, max=12
original_publication_day: min=0, max=31
default_chaptering_book_id: min=0, max=28572442


In [191]:
df["books_count"] = df["books_count"].astype(np.uint16)
df["reviews_count"] = df["reviews_count"].astype(np.uint32)
df["text_reviews_count"] = df["text_reviews_count"].astype(np.uint32)
df["best_book_id"] = df["best_book_id"].astype(np.uint32)
df["work_id"] = df["books_count"].astype(np.uint32)
df["n_ratings_1"] = df["n_ratings_1"].astype(np.uint32)
df["n_ratings_2"] = df["n_ratings_2"].astype(np.uint32)
df["n_ratings_3"] = df["n_ratings_3"].astype(np.uint32)
df["n_ratings_4"] = df["n_ratings_4"].astype(np.uint32)
df["n_ratings_5"] = df["n_ratings_5"].astype(np.uint32)
df["n_ratings_total"] = df["n_ratings_total"].astype(np.uint32)

df["original_publication_month"] = df["original_publication_month"].astype(np.uint8)
df["original_publication_day"] = df["original_publication_day"].astype(np.uint8)
df["original_publication_year"] = df["original_publication_year"].astype(np.uint16)
df["default_chaptering_book_id"] = df["default_chaptering_book_id"].astype(np.uint32)

df["original_title"] = df["original_title"].astype("string")
df["media_type"] = df["media_type"].astype("category")

In [192]:
df.loc[opm_nan, "original_publication_month"] = np.nan
df.loc[opd_nan, "original_publication_day"] = np.nan
df.loc[opy_nan, "original_publication_year"] = np.nan

df.loc[ot_nan, "original_title"] = np.nan
df.loc[dcbi_nan, "default_chaptering_book_id"] = np.nan

df.loc[mt_nan, "media_type"] = np.nan

df.isna().sum()

books_count                         0
reviews_count                       0
original_publication_month     486516
text_reviews_count                  0
best_book_id                        0
original_publication_year      252575
original_title                 875056
default_chaptering_book_id    1521302
original_publication_day       562502
media_type                     414096
work_id                             0
n_ratings_5                         0
n_ratings_4                         0
n_ratings_3                         0
n_ratings_2                         0
n_ratings_1                         0
n_ratings_total                     0
dtype: int64

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521962 entries, 0 to 1521961
Data columns (total 17 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   books_count                 1521962 non-null  uint16  
 1   reviews_count               1521962 non-null  uint32  
 2   original_publication_month  1035446 non-null  float64 
 3   text_reviews_count          1521962 non-null  uint32  
 4   best_book_id                1521962 non-null  uint32  
 5   original_publication_year   1269387 non-null  float64 
 6   original_title              646906 non-null   string  
 7   default_chaptering_book_id  660 non-null      float64 
 8   original_publication_day    959460 non-null   float64 
 9   media_type                  1107866 non-null  category
 10  work_id                     1521962 non-null  uint32  
 11  n_ratings_5                 1521962 non-null  uint32  
 12  n_ratings_4                 1521962 non-nu